<a href="https://colab.research.google.com/github/raranj/DeepLIFT_Cancer_TFs/blob/main/ToilNormCancerModelDeepLIFT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%tensorflow_version 1.x 

In [ ]:
from google.colab import drive
import pandas as pd
import tensorflow as tf
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt

drive.mount('/content/drive')
!ls "/content/drive/My Drive"

In [ ]:
# This function computes deepLIFT scores for each TF using
# gene expression numbers for tumor cells and corresponding
# normal tissue cells. It then plots the scores and prints the
# top n scores. normaldf is a dataframe for nomal cells,
# cancerdf is a dataframe for tumor cells, all_norm_df
# is the mean gene expression of all TFs across all normal tissues
def get_norm_cancer_scores(normaldf, cancerdf, all_norm_df):
  #drop columns that have a low mean (< 0.1) in either table to reduce noise
  n_mean = normaldf.mean(axis=0)
  c_mean = cancerdf.mean(axis=0)
  ncols = normaldf.shape[1]
  col_list = []
  for i in range(0, ncols): 
    if (n_mean[i] < .1) or c_mean[i] < 0.1:
      print("Dropping column %s" %normaldf.columns[i])
      col_list.append(i)

  # This will modify the argument in the caller to remove these columns
  if col_list:
    normaldf.drop(normaldf.columns[col_list], axis=1, inplace=True)
    cancerdf.drop(cancerdf.columns[col_list], axis=1, inplace=True)
    all_norm_df.drop(all_norm_df.columns[col_list], axis=1, inplace=True)

  #make a copy to avoid changing the original dataframe
  normal_df = normaldf.copy()
  cancer_df = cancerdf.copy()

  mean = normal_df.mean(axis=0)
  ref = mean.to_numpy()
  an_mean = all_norm_df.mean(axis=0)
  an_ref = an_mean.to_numpy()

  normal_df['cell_type'] = 1
  cancer_df['cell_type'] = 2

  cell_dfs = [normal_df, cancer_df] 
  X_data = pd.concat(cell_dfs) #one table that merges both types of cells

  Y_data = X_data.pop('cell_type') #removing column used for training

  #convert to numpy to be used for MLP model
  Y_np = Y_data.to_numpy() 
  X_np = X_data.to_numpy()

  num_patients = X_data.shape[0]
  num_TFs = X_data.shape[1]

  print("Number of Patients, TFs:")
  print(num_patients, num_TFs)

  model = tf.keras.Sequential() #Sequential model
  model.add(tf.keras.layers.Dense(256, input_shape=(num_TFs,))) #256 neurons, input has 1551 transcription factors in each row
  model.add(tf.keras.layers.Dense(128, activation = 'relu')) #hidden layer, 128 neurons, ReLU activation function
  model.add(tf.keras.layers.Dense(3, activation = 'softmax')) #output layer, normal or cancer types, softmax activation function
  model.compile(optimizer = 'adam', loss ='sparse_categorical_crossentropy', metrics = ['accuracy'])  

  #reshape single dimensional numpy array to array of patients vs. TFs
  X_np_i = X_np.reshape(num_patients, num_TFs).astype("float32") 
  Y_np_i = Y_np.reshape(num_patients, 1).astype("float32")

  #random_state 42 makes train_test_split deterministic, same every time
  X_train, X_test, Y_train, Y_test = train_test_split(X_np_i, Y_np_i, test_size=0.2, random_state=42)

  model.fit(X_train, Y_train, epochs = 10, validation_data=(X_test, Y_test))

  json_config = model.to_json()

  with open('TCGA_tissue_model.json', 'w') as json_file: 
    json_file.write(json_config)

  model.save_weights("TCGA_tissue_weights.h5")

  from deeplift.layers import NonlinearMxtsMode 
  import deeplift.conversion.kerasapi_conversion as kc #converts keras model to deepLIFT model
  from collections import OrderedDict

  #convert model to deepLIFT model
  method_to_model = OrderedDict()
  for method_name, nonlinear_mxts_mode in [
      ('revealcancel_all_layers', NonlinearMxtsMode.RevealCancel)]:
      method_to_model[method_name] = kc.convert_model_from_saved_files(
        h5_file = "TCGA_tissue_weights.h5",
        json_file = "TCGA_tissue_model.json",
        nonlinear_mxts_mode = nonlinear_mxts_mode)  
      
  #make sure predictions of the deepLIFT model are the same as the original model
  from deeplift.util import compile_func

  model_to_test = method_to_model['revealcancel_all_layers'] #revealcancel: method in deepLIFT
  deeplift_prediction_func = compile_func([model_to_test.get_layers()[0].get_activation_vars()], #0 is input layer, -1 is last layer
                                         model_to_test.get_layers()[-1].get_activation_vars())
  original_model_predictions = model.predict(X_np_i, batch_size = 200)
  converted_model_predictions = deeplift.util.run_function_in_batches(
                                input_data_list=[X_np_i],
                                func=deeplift_prediction_func,
                                batch_size=200,
                                progress_update=None)
  print("maximum difference in predictions:",np.max(np.array(converted_model_predictions)-np.array(original_model_predictions)))
  assert np.max(np.array(converted_model_predictions)-np.array(original_model_predictions)) < 10**-5
  predictions = converted_model_predictions 

  method_to_scoring_func = OrderedDict()
  for method, model in method_to_model.items(): #get scoring function for deepLIFT method
    print("Compiling scoring function for: " + method) 
    method_to_scoring_func[method] = model.get_target_contribs_func(find_scores_layer_idx=0, #get scores, 0 refers to input layer
                                                                    target_layer_idx=-2) #for softmax, should be -2 per https://github.com/kundajelab/deeplift/blob/master/README.md 

  n_normal = normal_df.shape[0]
  n_cancer = cancer_df.shape[0]

  #splitting the table by into normal and cancer rows
  X_np_normal = X_np_i[0:n_normal, :]
  X_np_cancer = X_np_i[n_normal:n_normal+n_cancer, :] 

  #call scoring functions to get scores
  method_to_task_to_scores = OrderedDict()
  for method_name, score_func in method_to_scoring_func.items():
    print("on method",method_name)
    method_to_task_to_scores[method_name] = OrderedDict()

    task_idx = 1 #1 is normal
    nscores = np.array(score_func( 
                    task_idx = task_idx,
                    input_data_list= [X_np_normal],
                    input_references_list=[an_ref], # normal tissue is reference
                    batch_size = 200,
                    progress_update = None))
    normal_scores = np.sum(nscores, axis = 0)

    task_idx = 2 #2 is cancer
    cscores = np.array(score_func( 
                    task_idx = task_idx,
                    input_data_list= [X_np_cancer],
                    input_references_list=[ref], # normal tissue is reference
                    batch_size = 200,
                    progress_update = None))
    cancer_scores = np.sum(cscores, axis = 0)

    cnscores = np.array(score_func( 
                    task_idx = task_idx,
                    input_data_list= [X_np_cancer],
                    input_references_list=[an_ref], # all normal tissue mean is reference
                    #input_references_list=[np.zeros_like(X_np_cancer)],
                    batch_size = 200,
                    progress_update = None))
    cancer_nscores = np.sum(cnscores, axis = 0)

  return normal_scores, cancer_scores, cancer_nscores

def get_aggregate_norm_cancer_scores(n_df, c_df, an_df):
  first_time = 1
  # copy every time as get_norm_cancer_scores may drop some columns
  all_norm_df = an_df.copy() 
  for i in range(0,10):
    normal, cancer, cancer_anref = get_norm_cancer_scores(n_df, c_df, all_norm_df)
    if (first_time == 1):
      normal_agg = normal
      cancer_agg = cancer
      cancer_anref_agg = cancer_anref
      first_time = 0
    else:
      normal_agg = np.add(normal_agg, normal)
      cancer_agg = np.add(cancer_agg, cancer)
      cancer_anref_agg = np.add(cancer_anref_agg, cancer_anref)

  print("Completed aggregate scores")
  return normal_agg, cancer_agg, cancer_anref_agg

# reove outliers - anything in the column outside of 1.5*IQR from the
# first and thirs quartile is an outlier.  Remove outliers and return
# the mean of the remaining values
def remove_outliers_and_get_mean(df_in):
    q1 = df_in.quantile(0.25)
    q3 = df_in.quantile(0.75)
    iqr = q3-q1 #Interquartile range
    fence_low  = q1-1.5*iqr
    fence_high = q3+1.5*iqr
    df_out = df_in.loc[(df_in > fence_low) & (df_in < fence_high)]
    return df_out.mean()

def plot_scores(scores, title):
  #plot TF versus score for normal
  plt.style.use('seaborn-whitegrid')
  plt.title(title)
  plt.plot(scores, 'v', color = 'black')
  plt.xlabel('transcription factor')
  plt.ylabel('DeepLIFT score')
  plt.show


def print_scores(normal_scores, cancer_scores, cancer_nscores, X_normal_data, X_cancer_data, TFs, n, tissue):
  print(tissue + " normal:")
  max_normal_r = np.argsort(normal_scores)[-n:]
  max_normal = np.flip(max_normal_r) # reverse it so index of highest value is first
  print("Indices:")
  print(max_normal)
  print("Aggregate scores")
  print(normal_scores[max_normal]) #prints the n highest aggregate scores in cancer_scores
  print("Transcription factors:")
  TF_names = TFs.columns[max_normal]
  print(TF_names)
  for name in TF_names:
    #cmean = X_cancer_data[name].mean()
    #nmean = X_normal_data[name].mean()
    cmean = remove_outliers_and_get_mean(X_cancer_data[name])
    nmean = remove_outliers_and_get_mean(X_normal_data[name])
    print("%s mean expression (cancer, normal): (%s %s)" %(name,cmean,nmean))
  print("")
  
  print(tissue + " cancer:")
  max_cancer_r = np.argsort(cancer_scores)[-n:]
  max_cancer = np.flip(max_cancer_r) # reverse it so index of highest value is first
  print("Indices:")
  print(max_cancer)
  print("Aggregate scores")
  print(cancer_scores[max_cancer]) #prints the n highest aggregate scores in cancer_scores
  print("Transcription factors:")
  TF_names = TFs.columns[max_cancer]
  print(TF_names)
  for name in TF_names:
    #cmean = X_cancer_data[name].mean()
    #nmean = X_normal_data[name].mean()
    cmean = remove_outliers_and_get_mean(X_cancer_data[name])
    nmean =  remove_outliers_and_get_mean(X_normal_data[name])
    print("%s mean expression (cancer, normal): (%s %s)" %(name,cmean,nmean))
  print("")  

  print(tissue + " cancer (all normal ref):")
  max_cancer_r = np.argsort(cancer_nscores)[-n:]
  max_cancer = np.flip(max_cancer_r) # reverse it so index of highest value is first
  print("Indices:")
  print(max_cancer)
  print("Aggregate scores")
  print(cancer_nscores[max_cancer]) #prints the n highest aggregate scores in cancer_scores
  print("Transcription factors:")
  TF_names = TFs.columns[max_cancer]
  print(TF_names)
  for name in TF_names:
    #cmean = X_cancer_data[name].mean()
    #nmean = X_normal_data[name].mean()
    cmean = remove_outliers_and_get_mean(X_cancer_data[name])
    nmean =  remove_outliers_and_get_mean(X_normal_data[name])
    print("%s mean expression (cancer, normal): (%s %s)" %(name,cmean,nmean))
  print("")

In [ ]:
#read data from file and return. dataframe with TFs as columns and patients as rows.
def get_tissue_data_df_t(filename, TrFa_df):
  data_df = pd.read_csv(filename, sep = '\\t')

  data_df.drop('Entrez_Gene_Id', axis=1, inplace=True)
  data_df.rename(columns={'Hugo_Symbol': 'sample'}, inplace=True)

  data_df_m = pd.merge(data_df, TrFa_df, on = 'sample', how = 'inner') #intersection of transcription factors in file and TF list

  data_df_m = data_df_m.set_index('sample') #set index so that TFs become column names after transpose 
  data_df_m = data_df_m.sort_index(axis=0) #sort alphabetically by TF names
  data_df_t = data_df_m.transpose() #TFs become columns, and patients become rows

  return data_df_t

In [ ]:
!pip install deeplift
import deeplift
!pip install scikit-learn
from sklearn.model_selection import train_test_split

In [ ]:
!cp "/content/drive/My Drive/list of transcription factors.txt" "TF_List"

In [ ]:
TrFa_df = pd.read_csv("TF_List", sep = '\\t')

In [ ]:
!cp "/content/drive/My Drive/TOIL_lung_TCGA.txt" "Lung_Cancer"
Lung_cancer_df_t = get_tissue_data_df_t("Lung_Cancer", TrFa_df)

In [ ]:
!cp "/content/drive/My Drive/TOIL_lung_GTEX.txt" "Lung_normal"
Lung_normal_df_t = get_tissue_data_df_t("Lung_normal", TrFa_df)

In [ ]:
!cp "/content/drive/My Drive/TOIL_colon_TCGA.txt" "Colon_Cancer"
Colon_cancer_df_t = get_tissue_data_df_t("Colon_Cancer", TrFa_df)

In [ ]:
!cp "/content/drive/My Drive/TOIL_colon_GTEX.txt" "Colon_Normal"
Colon_normal_df_t = get_tissue_data_df_t("Colon_Normal", TrFa_df)

In [ ]:
!cp "/content/drive/My Drive/TOIL_breast_TCGA.txt" "Breast_Cancer"
Breast_cancer_df_t = get_tissue_data_df_t("Breast_Cancer", TrFa_df)

In [ ]:
!cp "/content/drive/My Drive/TOIL_breast_GTEX.txt" "Breast_Normal"
Breast_normal_df_t = get_tissue_data_df_t("Breast_Normal", TrFa_df)

In [ ]:
!cp "/content/drive/My Drive/TOIL_thyroid_TCGA.txt" "Thyroid_Cancer"
Thyroid_cancer_df_t = get_tissue_data_df_t("Thyroid_Cancer", TrFa_df)

In [ ]:
!cp "/content/drive/My Drive/TOIL_thyroid_GTEX.txt" "Thyroid_Normal"
Thyroid_normal_df_t = get_tissue_data_df_t("Thyroid_Normal", TrFa_df)

In [ ]:
!cp "/content/drive/My Drive/TOIL_stomach_TCGA.txt" "Stomach_Cancer"
Stomach_cancer_df_t = get_tissue_data_df_t("Stomach_Cancer", TrFa_df)

In [ ]:
!cp "/content/drive/My Drive/TOIL_stomach_GTEX.txt" "Stomach_Normal"
Stomach_normal_df_t = get_tissue_data_df_t("Stomach_Normal", TrFa_df)

In [ ]:
!cp "/content/drive/My Drive/TOIL_prostate_TCGA.txt" "Prostate_Cancer"
Prostate_cancer_df_t = get_tissue_data_df_t("Prostate_Cancer", TrFa_df)

In [ ]:
!cp "/content/drive/My Drive/TOIL_prostate_GTEX.txt" "Prostate_Normal"
Prostate_normal_df_t = get_tissue_data_df_t("Prostate_Normal", TrFa_df)

In [ ]:
!cp "/content/drive/My Drive/TOIL_esophagus_TCGA.txt" "Esophagus_Cancer"
Esophagus_cancer_df_t = get_tissue_data_df_t("Esophagus_Cancer", TrFa_df)

In [ ]:
!cp "/content/drive/My Drive/TOIL_esophagus_GTEX.txt" "Esophagus_Normal"
Esophagus_normal_df_t = get_tissue_data_df_t("Esophagus_Normal", TrFa_df)

In [ ]:
!cp "/content/drive/My Drive/TOIL_liver_TCGA.txt" "Liver_Cancer"
Liver_cancer_df_t = get_tissue_data_df_t("Liver_Cancer", TrFa_df)

In [ ]:
!cp "/content/drive/My Drive/TOIL_liver_GTEX.txt" "Liver_Normal"
Liver_normal_df_t = get_tissue_data_df_t("Liver_Normal", TrFa_df)

In [ ]:
#get TFs that. are common across all cancers and normal tissues
common_cols = Lung_cancer_df_t.columns & Lung_normal_df_t.columns & \
              Colon_cancer_df_t.columns & Colon_normal_df_t.columns & \
              Breast_cancer_df_t.columns & Breast_normal_df_t.columns & \
              Thyroid_cancer_df_t.columns & Thyroid_normal_df_t.columns & \
              Stomach_cancer_df_t.columns & Stomach_normal_df_t.columns & \
              Prostate_cancer_df_t.columns & Prostate_normal_df_t.columns & \
              Esophagus_cancer_df_t.columns & Esophagus_normal_df_t.columns & \
              Liver_cancer_df_t.columns & Liver_normal_df_t.columns

In [ ]:
# Retain omly the common columns
Lung_cancer_df_t = Lung_cancer_df_t[common_cols]
Lung_normal_df_t = Lung_normal_df_t[common_cols]
Colon_cancer_df_t = Colon_cancer_df_t[common_cols]
Colon_normal_df_t = Colon_normal_df_t[common_cols]
Breast_cancer_df_t = Breast_cancer_df_t[common_cols]
Breast_normal_df_t = Breast_normal_df_t[common_cols]
Thyroid_cancer_df_t = Thyroid_cancer_df_t[common_cols]
Thyroid_normal_df_t = Thyroid_normal_df_t[common_cols]
Stomach_cancer_df_t = Stomach_cancer_df_t[common_cols]
Stomach_normal_df_t = Stomach_normal_df_t[common_cols]
Prostate_cancer_df_t = Prostate_cancer_df_t[common_cols]
Prostate_normal_df_t = Prostate_normal_df_t[common_cols]
Esophagus_cancer_df_t = Esophagus_cancer_df_t[common_cols]
Esophagus_normal_df_t = Esophagus_normal_df_t[common_cols]
Liver_cancer_df_t = Liver_cancer_df_t[common_cols]
Liver_normal_df_t = Liver_normal_df_t[common_cols]

In [ ]:
#Get concatenated table of all normal tissue before adding the 'cancer_type' column.
#This will be used to get the mean of TF expression values across all tissue types,
#which can then be used as the DeepLIFT reference.
ref_normal_dfs = [Lung_normal_df_t, Colon_normal_df_t, Breast_normal_df_t, Thyroid_normal_df_t, Stomach_normal_df_t, Prostate_normal_df_t, Esophagus_normal_df_t, Liver_normal_df_t]
ref_normal_df_t = pd.concat(ref_normal_dfs) #one table that merges all types of tissues

In [ ]:
lung_nscore, lung_cscore, lung_ncscore = get_aggregate_norm_cancer_scores(Lung_normal_df_t, Lung_cancer_df_t, ref_normal_df_t)
colon_nscore, colon_cscore, colon_ncscore = get_aggregate_norm_cancer_scores(Colon_normal_df_t, Colon_cancer_df_t, ref_normal_df_t)
breast_nscore, breast_cscore, breast_ncscore = get_aggregate_norm_cancer_scores(Breast_normal_df_t, Breast_cancer_df_t, ref_normal_df_t)
thyroid_nscore, thyroid_cscore, thyroid_ncscore = get_aggregate_norm_cancer_scores(Thyroid_normal_df_t, Thyroid_cancer_df_t, ref_normal_df_t)
stomach_nscore, stomach_cscore, stomach_ncscore = get_aggregate_norm_cancer_scores(Stomach_normal_df_t, Stomach_cancer_df_t, ref_normal_df_t)
prostate_nscore, prostate_cscore, prostate_ncscore = get_aggregate_norm_cancer_scores(Prostate_normal_df_t, Prostate_cancer_df_t, ref_normal_df_t)
esophagus_nscore, esophagus_cscore, esophagus_ncscore = get_aggregate_norm_cancer_scores(Esophagus_normal_df_t, Esophagus_cancer_df_t, ref_normal_df_t)
liver_nscore, liver_cscore, liver_ncscore = get_aggregate_norm_cancer_scores(Liver_normal_df_t, Liver_cancer_df_t, ref_normal_df_t)

In [ ]:
n = 10

In [ ]:
print_scores(lung_nscore,lung_cscore,lung_ncscore,Lung_normal_df_t, Lung_cancer_df_t,Lung_cancer_df_t, n,"Lung")

In [ ]:
print_scores(colon_nscore,colon_cscore,colon_ncscore,Colon_normal_df_t, Colon_cancer_df_t,Colon_cancer_df_t, n,"Colon")

In [ ]:
print_scores(breast_nscore,breast_cscore,breast_ncscore,Breast_normal_df_t, Breast_cancer_df_t,Breast_cancer_df_t, n,"Breast")

In [ ]:
print_scores(thyroid_nscore,thyroid_cscore,thyroid_ncscore,Thyroid_normal_df_t, Thyroid_cancer_df_t, Thyroid_cancer_df_t, n,"Thyroid")

In [ ]:
print_scores(stomach_nscore,stomach_cscore,stomach_ncscore,Stomach_normal_df_t,Stomach_cancer_df_t, Stomach_cancer_df_t, n,"Stomach")

In [ ]:
print_scores(prostate_nscore,prostate_cscore,prostate_ncscore,Prostate_normal_df_t,Prostate_cancer_df_t, Prostate_cancer_df_t, n,"Prostate")

In [ ]:
print_scores(esophagus_nscore,esophagus_cscore,esophagus_ncscore,Esophagus_normal_df_t,Esophagus_cancer_df_t,Esophagus_cancer_df_t, n,"Esophagus")

In [ ]:
print_scores(liver_nscore,liver_cscore,liver_ncscore,Liver_normal_df_t,Liver_cancer_df_t,Liver_cancer_df_t, n,"Liver")

In [ ]:
plot_scores(lung_nscore, "Lung normal")

In [ ]:
plot_scores(lung_cscore, "Lung cancer")

In [ ]:
plot_scores(lung_ncscore, "Lung cancer (all normal reference)")

In [ ]:
plot_scores(colon_nscore, "Colon normal")

In [ ]:
plot_scores(colon_cscore, "Colon cancer")

In [ ]:
plot_scores(colon_ncscore, "Colon cancer (all normal reference)")

In [ ]:
plot_scores(breast_nscore,"Breast normal")

In [ ]:
plot_scores(breast_cscore,"Breast cancer")

In [ ]:
plot_scores(breast_nscore,"Breast cancer (all normal reference)")

In [ ]:
plot_scores(thyroid_nscore,"Thyroid normal")

In [ ]:
plot_scores(thyroid_cscore,"Thyroid cancer")

In [ ]:
plot_scores(thyroid_ncscore,"Thyroid cancer (all normal reference)")

In [ ]:
plot_scores(stomach_nscore,"Stomach normal")

In [ ]:
plot_scores(stomach_cscore,"Stomach cancer")

In [ ]:
plot_scores(stomach_ncscore,"Stomach cancer (all normal reference)")

In [ ]:
plot_scores(prostate_nscore,"Prostate normal")

In [ ]:
plot_scores(prostate_cscore,"Prostate cancer")

In [ ]:
plot_scores(prostate_ncscore,"Prostate cancer (all normal reference)")

In [ ]:
plot_scores(esophagus_nscore,"Esophagus normal")

In [ ]:
plot_scores(esophagus_cscore,"Esophagus cancer")

In [ ]:
plot_scores(esophagus_ncscore,"Esophagus cancer (all normal reference)")

In [ ]:
plot_scores(liver_nscore,"Liver normal")

In [ ]:
plot_scores(liver_cscore,"Liver cancer")

In [ ]:
plot_scores(liver_ncscore,"Liver cancer (all normal reference)")